In [9]:
import pandas as pd

path = '/home/gridsan/afogelson/osfm/scripts/urop_samples/uniform_sample/uniform_urop_sample_alex_labeled.csv'
df = pd.read_csv(path)

manual_column_name = 'alex2'

if (manual_column_name not in set(df.columns)):
    df[manual_column_name] = [None for _ in range(len(df))]

options = {str(idx): el for idx, el in enumerate(['background', 'motivation', 'extends', 'similarities', 'motivation', 'uses', 'differences', 'future work'])} | {'*': '*'}
print(options)

{'0': 'background', '1': 'motivation', '2': 'extends', '3': 'similarities', '4': 'motivation', '5': 'uses', '6': 'differences', '7': 'future work', '*': '*'}


In [21]:
mask = df[manual_column_name].isna()
print(f"{mask.sum()}/{len(mask)} left.", flush = True)

38/201 left.


In [24]:
import random
from IPython.display import clear_output


mask = df[manual_column_name].isna()
while mask.sum() > 0:
    idx = random.choice(df[mask].index)
    
    print(f"{mask.sum()}/{len(mask)} left.", flush = True)
    print(options, flush = True)
    print(f"Model Key:{df.at[idx,'modelKey']}", flush = True)
    print(df.at[idx, 'multisentence'], flush = True)

    response = input()
    
    if (response == 'exit'):
        df.to_csv(path)
        clear_output()
        break
    
    df.loc[idx, manual_column_name] = options.get(response)
    mask = df[manual_column_name].isna()
    clear_output()
    df.to_csv(path, index = False)
    

In [25]:
df.to_csv(path, index = False)

In [31]:
df[df['alex2'] == "*"]

,Unnamed: 0.4,Unnamed: 0.3,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,sentence,mcllm,modelKey,paperId,modelId_x,...,mc_reduced,urop_sentence,urop,urop_assignee,gpt,mcllmr,mcllm_binary,alex,multisentence,alex2
83,83,83,83,83,83,they are calculated as follows:\n\n\n\[\begin{...,uses,573_yolov3,bc0fda8fd4cb15410e40f0a07c7c5f2fcba20cbe,ebc96892b9bcbf007be9a1d7844e4b09fde9d961,...,uses,they are calculated as follows:\n\n\[\begin{sp...,Select,NaN,background,uses,uses,*,recognition performance of he-yolo. they are c...,*
126,126,126,126,126,126,"for a comparative analysis, we experiment with...",uses,582_bart-large,021c4ba0cbc8ebe9290cf74ef8b3d653f625bd54,395de0bd3837fdf4b4b5e5f04835bcc69c279481,...,uses,"for a comparative analysis, we experiment with...",Differences,Selinna,differences,background,uses,NaN,## baseline methods and comparisons\n\nwe con...,*
135,135,135,135,135,135,this example is adapted from test cases for sh...,uses,521_roberta_large,6ab8aca8f631f42760a86cc614dfd7208b3fe58e,077f8329a7b6fa3b7c877a57b81eb6c18b5f87de,...,extends,this example is adapted from test cases for sh...,Background,Selinna,background,background,extends,NaN,## 2. running example\n\nfigure 1 shows the m...,*
184,184,184,184,184,184,we basically follow the original paper <cite>d...,uses,1064_bert-large,ee8d05579b258a287a700e8b745959cba4713a36,df2b0e26d0599ce3e70df8a9da02e51594e0e992,...,extends,we basically follow the original paper <cite>d...,Motivation,Selinna,extends,differences,extends,extends,"hyper-parametersfor pretraining, we train the ...",*
189,189,189,189,189,189,4.1)\n\n\n### encoding code cells with attenti...,extends,1064_bert-large,75e36bb95023e55f7dec95d1af557e219ba3d349,df2b0e26d0599ce3e70df8a9da02e51594e0e992,...,extends,4.1)\n\n### encoding code cells with attention...,Extends,Denis,extends,extends,extends,NaN,"in practice, we also use connections between [...",*


# Matching single sentences with their context from another dataframe

In [ ]:
new_results = pd.read_csv('/home/gridsan/afogelson/osfm/paper_analysis_toolkit/results/citations/results_2024-06-13 09:43:52_worker1of1.csv')

In [70]:
def get_multisentence(row):
    sentence, paperId, modelKey = row['sentence'], row['paperId'], row['modelKey']
    mask = (new_results['paperId'] == paperId) & (new_results['modelKey'] == modelKey) & new_results['sentence'].apply(lambda s: s.find(sentence[:17]) >= 0)
    if mask.sum() == 1:
        return new_results[mask]['sentence'].iloc[0]
    else:
        return None


In [71]:
df['multisentence'] = df.apply(get_multisentence, axis = 1)


19

In [163]:
missing = df['multisentence'].isna()
print(missing.sum())
idx = 0
target_sentence, modelKey, paperId, index = df[missing]['sentence'].iloc[idx], df[missing]['modelKey'].iloc[idx], df[missing]['paperId'].iloc[idx], df[missing].index[idx]

print(target_sentence)
valid_results = new_results[(new_results['modelKey'] == modelKey) & (new_results['paperId'] == paperId) & new_results['sentence'].apply(lambda s: s.find(target_sentence[:5]) >=0)]

valid_results

0


IndexError: single positional indexer is out-of-bounds

In [161]:
print(valid_results['sentence'].iloc[2])

\begin{table}
\begin{tabular}{c c c c c c} \hline \hline \multirow{2}{*}{**method**} & \multicolumn{4}{c}{**noise level**} \\ \cline{2-5}  & 0\% & 10\% & 20\% & 40\% & 60\% \\ \cline{2-5}  & top-1(top-5) & top-1(top-5) & top-1(top-5) & top-1(top-5) \\ \hline bbn (zhou et al, 2020) & 63.11(**87.06**) & 62.03(**86.79**) & 60.03(**85.73**) & 55.59(**83.68**) & 50.22(80.47) \\ ldam (cao et al, 2019) & **64.25**(86.82) & **62.71**(86.19) & **60.69**(85.18) & **56.29**(82.53) & **50.79**(79.52) \\ decoupling-ncm (kang et al, 2020) & 62.54(85.59) & 60.85(85.61) & 58.94(84.71) & 54.86(82.58) & 50.09(79.76) \\ decoupling-crt (kang et al, 2020) & 62.89(86.16) & 61.86(86.53) & 59.99(85.41) & 55.80(83.29) & 51.82(**81.40**) \\ decoupling-lws (kang et al, 2020) & 61.87(85.75) & 60.42(85.96) & 58.61(84.63) & 54.30(82.20) & 49.61(79.50) \\ \hline \hline \end{tabular}
\end{table}
table 11: optimal hyperparameter settings for different controlled synthetic label noise on noisywikihow. \begin{table}
\be

In [162]:
found_multisentence = valid_results['sentence'].iloc[2]
print(found_multisentence)
df.at[index,'multisentence'] = found_multisentence
print(df.at[index,'multisentence'])

\begin{table}
\begin{tabular}{c c c c c c} \hline \hline \multirow{2}{*}{**method**} & \multicolumn{4}{c}{**noise level**} \\ \cline{2-5}  & 0\% & 10\% & 20\% & 40\% & 60\% \\ \cline{2-5}  & top-1(top-5) & top-1(top-5) & top-1(top-5) & top-1(top-5) \\ \hline bbn (zhou et al, 2020) & 63.11(**87.06**) & 62.03(**86.79**) & 60.03(**85.73**) & 55.59(**83.68**) & 50.22(80.47) \\ ldam (cao et al, 2019) & **64.25**(86.82) & **62.71**(86.19) & **60.69**(85.18) & **56.29**(82.53) & **50.79**(79.52) \\ decoupling-ncm (kang et al, 2020) & 62.54(85.59) & 60.85(85.61) & 58.94(84.71) & 54.86(82.58) & 50.09(79.76) \\ decoupling-crt (kang et al, 2020) & 62.89(86.16) & 61.86(86.53) & 59.99(85.41) & 55.80(83.29) & 51.82(**81.40**) \\ decoupling-lws (kang et al, 2020) & 61.87(85.75) & 60.42(85.96) & 58.61(84.63) & 54.30(82.20) & 49.61(79.50) \\ \hline \hline \end{tabular}
\end{table}
table 11: optimal hyperparameter settings for different controlled synthetic label noise on noisywikihow. \begin{table}
\be

# Random Other Debugging, can delete later

In [8]:
string = '{"explanation": "The citation indicates the use of a specific convolutional neural network (convnet) architecture called spp-net for pyramid pooling, but it does not explicitly mention the model itself being used for any specific task or process in the given context.", "classification": "context"}'
from typing import Optional
import json

def stripJSON(text: Optional[str], added_bracket = False) -> Optional[dict]:
        if (text is None):
            return None
        
        text = text.replace('\(', '(').replace('\)', ')') #Mistral likes to add escape sequences, for some unknown reason
        # a very manual way of finding our JSON string within the output
        all_open_brackets = [i for i, ltr in enumerate(text) if ltr == '{']
        obj = None
        for start in all_open_brackets:
            print(start)
            balance_counter = 1
            for offset, chr in enumerate(text[start + 1:], start = 1):
                balance_counter += (1 if chr == '{' else -1 if chr == '}' else 0)
                if (balance_counter == 0):
                    try:
                        obj = json.loads(text[start: start + offset + 1])
                    except:
                        pass
                    break
            if (obj is not None):
                break
            
        # sometimes we miss the first or last bracket (dumb LLM), so we add it manually.
        #if not added_bracket:
        #    return (stripJSON('{' + text, added_bracket = True) or stripJSON(text + '}', added_bracket = True))
            
        return obj
    
print(stripJSON(string))

0
{'explanation': 'The citation indicates the use of a specific convolutional neural network (convnet) architecture called spp-net for pyramid pooling, but it does not explicitly mention the model itself being used for any specific task or process in the given context.', 'classification': 'context'}


In [5]:
json.loads(string)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)